# 🛰️ Pansharpening Results Visualization

A modern, interactive notebook for visualizing and comparing pansharpening results.

**Features:**
- Side-by-side comparison (MS, PAN, Fused)
- Interactive before/after slider
- Quality metrics (PSNR, SSIM, SAM)
- Histogram analysis
- Zoom comparison
- Export high-resolution images

## 1. Setup & Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches
from scipy.ndimage import zoom
import warnings
warnings.filterwarnings('ignore')

# ============================================
# SET YOUR PROJECT ROOT PATH HERE
# ============================================
PROJECT_ROOT = Path(r"D:\Udemy_Cour\Pancharping\pansharpening_project")

# Alternative: Auto-detect (uncomment if needed)
# PROJECT_ROOT = Path.cwd()
# if PROJECT_ROOT.name == 'notebooks':
#     PROJECT_ROOT = PROJECT_ROOT.parent

# Verify the path
if not (PROJECT_ROOT / "data").exists():
    print(f"❌ ERROR: Project root not found at: {PROJECT_ROOT}")
    print("   Please update PROJECT_ROOT in this cell!")
else:
    print(f"✅ Project root: {PROJECT_ROOT}")

sys.path.insert(0, str(PROJECT_ROOT))

# Try to import rasterio for GeoTIFF support
try:
    import rasterio
    HAS_RASTERIO = True
    print("✅ Rasterio available - GeoTIFF support enabled")
except ImportError:
    HAS_RASTERIO = False
    from PIL import Image
    print("⚠️ Rasterio not found - using PIL (limited GeoTIFF support)")

# Interactive widgets
try:
    from ipywidgets import interact, interactive, IntSlider, FloatSlider, Dropdown, HBox, VBox, Output
    import ipywidgets as widgets
    HAS_WIDGETS = True
    print("✅ ipywidgets available - Interactive features enabled")
except ImportError:
    HAS_WIDGETS = False
    print("⚠️ ipywidgets not found - Install with: pip install ipywidgets")

# Verify data files exist
print(f"\n📂 Data files:")
print(f"   ms.tif:  {'✅' if (PROJECT_ROOT / 'data' / 'ms.tif').exists() else '❌'}")
print(f"   pan.tif: {'✅' if (PROJECT_ROOT / 'data' / 'pan.tif').exists() else '❌'}")

## 2. Modern Theme Setup

In [ ]:
# Modern dark theme colors
COLORS = {
    'bg': '#1a1a2e',
    'panel': '#16213e',
    'accent': '#0f3460',
    'highlight': '#e94560',
    'text': '#eaeaea',
    'text_dim': '#a0a0a0',
    'success': '#00d9ff',
    'warning': '#ffd700',
    'ms_color': '#ff6b6b',
    'pan_color': '#4ecdc4',
    'fused_color': '#45b7d1',
}

def apply_modern_style():
    """Apply modern dark theme to matplotlib."""
    plt.style.use('dark_background')
    plt.rcParams.update({
        'figure.facecolor': COLORS['bg'],
        'axes.facecolor': COLORS['panel'],
        'axes.edgecolor': COLORS['accent'],
        'axes.labelcolor': COLORS['text'],
        'axes.titlecolor': COLORS['text'],
        'xtick.color': COLORS['text_dim'],
        'ytick.color': COLORS['text_dim'],
        'text.color': COLORS['text'],
        'grid.color': COLORS['accent'],
        'grid.alpha': 0.3,
        'font.size': 10,
        'axes.titlesize': 12,
        'figure.titlesize': 14,
    })

apply_modern_style()
print("🎨 Modern theme applied!")

## 3. Image Loading Functions

In [ ]:
def load_image(path):
    """Load image from file (supports GeoTIFF and common formats)."""
    path = Path(path)
    
    if not path.exists():
        raise FileNotFoundError(f"Image not found: {path}")
    
    if HAS_RASTERIO and path.suffix.lower() in ['.tif', '.tiff']:
        with rasterio.open(path) as src:
            img = src.read()
            if img.ndim == 3:
                img = np.transpose(img, (1, 2, 0))
            return img.astype(np.float32)
    else:
        img = np.array(Image.open(path))
        return img.astype(np.float32)


def normalize_for_display(img, percentile=(2, 98)):
    """Normalize image for display using percentile stretching."""
    img = img.copy()
    
    if img.ndim == 2:
        p_low, p_high = np.percentile(img, percentile)
        img = np.clip((img - p_low) / (p_high - p_low + 1e-8), 0, 1)
    else:
        for i in range(min(img.shape[-1], 3)):
            band = img[..., i]
            p_low, p_high = np.percentile(band, percentile)
            img[..., i] = np.clip((band - p_low) / (p_high - p_low + 1e-8), 0, 1)
    
    return img


def create_rgb_composite(ms_img, bands=(2, 1, 0)):
    """Create RGB composite from multispectral image."""
    if ms_img.ndim == 2:
        rgb = np.stack([ms_img] * 3, axis=-1)
    elif ms_img.shape[-1] >= 3:
        rgb = ms_img[..., list(bands)]
    else:
        rgb = np.stack([ms_img[..., 0]] * 3, axis=-1)
    
    return normalize_for_display(rgb)


print("✅ Image loading functions ready!")

## 4. Quality Metrics

In [ ]:
def calculate_psnr(img1, img2):
    """Calculate Peak Signal-to-Noise Ratio."""
    mse = np.mean((img1.astype(np.float64) - img2.astype(np.float64)) ** 2)
    if mse == 0:
        return float('inf')
    max_val = max(img1.max(), img2.max())
    return 10 * np.log10(max_val ** 2 / mse)


def calculate_ssim(img1, img2):
    """Calculate Structural Similarity Index (simplified)."""
    c1 = (0.01 * 255) ** 2
    c2 = (0.03 * 255) ** 2
    
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    
    if img1.ndim == 3:
        ssim_vals = []
        for i in range(img1.shape[-1]):
            mu1 = np.mean(img1[..., i])
            mu2 = np.mean(img2[..., i])
            sigma1 = np.std(img1[..., i])
            sigma2 = np.std(img2[..., i])
            sigma12 = np.mean((img1[..., i] - mu1) * (img2[..., i] - mu2))
            
            ssim = ((2 * mu1 * mu2 + c1) * (2 * sigma12 + c2)) / \
                   ((mu1**2 + mu2**2 + c1) * (sigma1**2 + sigma2**2 + c2))
            ssim_vals.append(ssim)
        return np.mean(ssim_vals)
    else:
        mu1, mu2 = np.mean(img1), np.mean(img2)
        sigma1, sigma2 = np.std(img1), np.std(img2)
        sigma12 = np.mean((img1 - mu1) * (img2 - mu2))
        return ((2 * mu1 * mu2 + c1) * (2 * sigma12 + c2)) / \
               ((mu1**2 + mu2**2 + c1) * (sigma1**2 + sigma2**2 + c2))


def calculate_sam(img1, img2):
    """Calculate Spectral Angle Mapper (in degrees)."""
    if img1.ndim != 3 or img2.ndim != 3:
        return None
    
    img1_flat = img1.reshape(-1, img1.shape[-1]).astype(np.float64)
    img2_flat = img2.reshape(-1, img2.shape[-1]).astype(np.float64)
    
    dot_product = np.sum(img1_flat * img2_flat, axis=1)
    norm1 = np.linalg.norm(img1_flat, axis=1)
    norm2 = np.linalg.norm(img2_flat, axis=1)
    
    cos_angle = dot_product / (norm1 * norm2 + 1e-8)
    cos_angle = np.clip(cos_angle, -1, 1)
    
    return np.mean(np.arccos(cos_angle)) * 180 / np.pi


def calculate_all_metrics(fused, reference):
    """Calculate all quality metrics."""
    metrics = {
        'PSNR': calculate_psnr(fused, reference),
        'SSIM': calculate_ssim(fused, reference),
    }
    sam = calculate_sam(fused, reference)
    if sam is not None:
        metrics['SAM'] = sam
    return metrics


print("✅ Quality metrics functions ready!")

## 5. Load Images

In [ ]:
# Define paths
MS_PATH = PROJECT_ROOT / "data" / "ms.tif"
PAN_PATH = PROJECT_ROOT / "data" / "pan.tif"

# Find fused image
FUSED_PATHS = [
    PROJECT_ROOT / "results" / "deep_learning" / "fused_pannet.tif",
    PROJECT_ROOT / "results" / "deep_learning" / "fused_panformer_lite.tif",
    PROJECT_ROOT / "results" / "classic" / "fused_sfim.tif",
    PROJECT_ROOT / "results" / "classic" / "fused_brovey.tif",
]

FUSED_PATH = None
for p in FUSED_PATHS:
    if p.exists():
        FUSED_PATH = p
        break

print("📂 Loading images...")
print(f"   MS:  {MS_PATH}")
print(f"   PAN: {PAN_PATH}")
print(f"   Fused: {FUSED_PATH if FUSED_PATH else 'Not found (will generate demo)'}")

In [ ]:
# Load images
ms_raw = load_image(MS_PATH)
pan_raw = load_image(PAN_PATH)

print(f"\n📊 Image Information:")
print(f"   MS shape:  {ms_raw.shape}")
print(f"   PAN shape: {pan_raw.shape}")

# Load or generate fused image
if FUSED_PATH and FUSED_PATH.exists():
    fused_raw = load_image(FUSED_PATH)
    print(f"   Fused shape: {fused_raw.shape}")
else:
    print("\n⚠️ No fused image found. Generating demo fusion (Brovey)...")
    
    # Upsample MS to PAN resolution
    scale_h = pan_raw.shape[0] / ms_raw.shape[0]
    scale_w = pan_raw.shape[1] / ms_raw.shape[1]
    
    if ms_raw.ndim == 2:
        ms_up = zoom(ms_raw, (scale_h, scale_w), order=1)
    else:
        ms_up = zoom(ms_raw, (scale_h, scale_w, 1), order=1)
    
    # Simple Brovey transform
    pan = pan_raw if pan_raw.ndim == 2 else pan_raw[..., 0]
    intensity = np.mean(ms_up, axis=-1) if ms_up.ndim == 3 else ms_up
    ratio = pan / (intensity + 1e-8)
    
    if ms_up.ndim == 3:
        fused_raw = ms_up * ratio[..., np.newaxis]
    else:
        fused_raw = ms_up * ratio
    
    print(f"   Generated fused shape: {fused_raw.shape}")

print("\n✅ Images loaded successfully!")

In [ ]:
# Prepare display images
ms_display = create_rgb_composite(ms_raw)
pan_display = normalize_for_display(pan_raw)
fused_display = create_rgb_composite(fused_raw)

# Upsample MS for comparison
if ms_display.shape[:2] != pan_display.shape[:2]:
    scale_h = pan_display.shape[0] / ms_display.shape[0]
    scale_w = pan_display.shape[1] / ms_display.shape[1]
    ms_display_up = zoom(ms_display, (scale_h, scale_w, 1), order=1)
else:
    ms_display_up = ms_display

print(f"Display shapes:")
print(f"   MS (upscaled): {ms_display_up.shape}")
print(f"   PAN: {pan_display.shape}")
print(f"   Fused: {fused_display.shape}")

---
## 6. 📊 Side-by-Side Comparison

In [ ]:
apply_modern_style()

fig = plt.figure(figsize=(16, 10))
fig.suptitle('🛰️ Pansharpening Results Comparison', fontsize=18, fontweight='bold',
             color=COLORS['success'], y=0.98)

gs = GridSpec(2, 3, figure=fig, height_ratios=[1, 0.12], hspace=0.25, wspace=0.15)

# Images
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])

# Display
ax1.imshow(ms_display_up)
ax1.set_title('📊 Multispectral (MS)', fontsize=14, pad=10, color=COLORS['ms_color'])
ax1.axis('off')
ax1.text(0.5, -0.08, 'Low Resolution • High Spectral', transform=ax1.transAxes,
         ha='center', fontsize=10, color=COLORS['text_dim'])

ax2.imshow(pan_display, cmap='gray')
ax2.set_title('📷 Panchromatic (PAN)', fontsize=14, pad=10, color=COLORS['pan_color'])
ax2.axis('off')
ax2.text(0.5, -0.08, 'High Resolution • Single Band', transform=ax2.transAxes,
         ha='center', fontsize=10, color=COLORS['text_dim'])

ax3.imshow(fused_display)
ax3.set_title('✨ Pansharpened Result', fontsize=14, pad=10, color=COLORS['success'])
ax3.axis('off')
ax3.text(0.5, -0.08, 'High Resolution • High Spectral', transform=ax3.transAxes,
         ha='center', fontsize=10, color=COLORS['success'])

# Metrics panel
ax_metrics = fig.add_subplot(gs[1, :])
ax_metrics.axis('off')

metrics = calculate_all_metrics(fused_display, ms_display_up)
metrics_text = f"📈 PSNR: {metrics['PSNR']:.2f} dB   |   🎯 SSIM: {metrics['SSIM']:.4f}"
if 'SAM' in metrics:
    metrics_text += f"   |   🌈 SAM: {metrics['SAM']:.2f}°"

ax_metrics.text(0.5, 0.5, metrics_text, transform=ax_metrics.transAxes,
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.8', facecolor=COLORS['accent'],
                          edgecolor=COLORS['success'], linewidth=2))

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'comparison_modern.png', dpi=150, 
            facecolor=COLORS['bg'], bbox_inches='tight')
print("💾 Saved: results/comparison_modern.png")
plt.show()

---
## 7. 🔄 Interactive Before/After Slider

In [ ]:
if HAS_WIDGETS:
    apply_modern_style()
    
    output = Output()
    
    def update_slider(position=50):
        with output:
            output.clear_output(wait=True)
            
            fig, ax = plt.subplots(figsize=(12, 10))
            fig.suptitle('🔄 Before / After Comparison', fontsize=16, 
                         fontweight='bold', color=COLORS['success'])
            
            # Create combined image
            before = ms_display_up
            after = fused_display
            combined = after.copy()
            split_x = int(combined.shape[1] * position / 100)
            combined[:, :split_x] = before[:, :split_x]
            
            ax.imshow(combined)
            ax.axvline(x=split_x, color=COLORS['highlight'], linewidth=3)
            ax.axis('off')
            
            # Labels
            ax.text(0.15, 0.95, '← BEFORE (MS)', transform=ax.transAxes,
                    fontsize=14, color=COLORS['warning'], fontweight='bold',
                    ha='center', va='top')
            ax.text(0.85, 0.95, 'AFTER (Fused) →', transform=ax.transAxes,
                    fontsize=14, color=COLORS['success'], fontweight='bold',
                    ha='center', va='top')
            
            plt.tight_layout()
            plt.show()
    
    slider = IntSlider(value=50, min=0, max=100, step=1,
                       description='Position:', style={'description_width': '80px'},
                       layout=widgets.Layout(width='80%'))
    
    interactive_plot = interactive(update_slider, position=slider)
    display(VBox([slider, output]))
    update_slider(50)
else:
    print("⚠️ Install ipywidgets for interactive slider: pip install ipywidgets")
    
    # Static version
    fig, ax = plt.subplots(figsize=(12, 10))
    combined = fused_display.copy()
    split_x = combined.shape[1] // 2
    combined[:, :split_x] = ms_display_up[:, :split_x]
    ax.imshow(combined)
    ax.axvline(x=split_x, color=COLORS['highlight'], linewidth=3)
    ax.axis('off')
    ax.set_title('Before (MS) | After (Fused)', fontsize=14)
    plt.show()

---
## 8. 📈 Detailed Analysis

In [ ]:
apply_modern_style()

fig = plt.figure(figsize=(18, 12))
fig.suptitle('📊 Detailed Pansharpening Analysis', fontsize=18,
             fontweight='bold', color=COLORS['success'])

gs = GridSpec(3, 4, figure=fig, hspace=0.35, wspace=0.3)

# Row 1: Images
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[0, 3])

ax1.imshow(ms_display_up)
ax1.set_title('MS (Upscaled)', fontsize=11, color=COLORS['ms_color'])
ax1.axis('off')

ax2.imshow(pan_display, cmap='gray')
ax2.set_title('PAN', fontsize=11, color=COLORS['pan_color'])
ax2.axis('off')

ax3.imshow(fused_display)
ax3.set_title('Fused', fontsize=11, color=COLORS['success'])
ax3.axis('off')

# Difference map
diff = np.abs(fused_display - ms_display_up)
diff_gray = np.mean(diff, axis=-1) if diff.ndim == 3 else diff
im4 = ax4.imshow(diff_gray, cmap='hot')
ax4.set_title('Difference Map', fontsize=11)
ax4.axis('off')
plt.colorbar(im4, ax=ax4, fraction=0.046, pad=0.04)

# Row 2: Histograms
ax5 = fig.add_subplot(gs[1, :2])
ax6 = fig.add_subplot(gs[1, 2:])

colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']
labels = ['Red', 'Green', 'Blue']

for i, (c, lbl) in enumerate(zip(colors, labels)):
    if i < ms_display_up.shape[-1]:
        ax5.hist(ms_display_up[..., i].ravel(), bins=100, alpha=0.5, color=c, label=lbl)
ax5.set_title('MS Histogram', fontsize=11, color=COLORS['ms_color'])
ax5.set_xlabel('Pixel Value')
ax5.set_ylabel('Frequency')
ax5.legend()
ax5.grid(True, alpha=0.3)

for i, (c, lbl) in enumerate(zip(colors, labels)):
    if i < fused_display.shape[-1]:
        ax6.hist(fused_display[..., i].ravel(), bins=100, alpha=0.5, color=c, label=lbl)
ax6.set_title('Fused Histogram', fontsize=11, color=COLORS['success'])
ax6.set_xlabel('Pixel Value')
ax6.set_ylabel('Frequency')
ax6.legend()
ax6.grid(True, alpha=0.3)

# Row 3: Profile and Metrics
ax7 = fig.add_subplot(gs[2, :2])
ax8 = fig.add_subplot(gs[2, 2:])

# Horizontal profile
mid_row = fused_display.shape[0] // 2
profile_ms = np.mean(ms_display_up[mid_row, :, :], axis=-1)
profile_fused = np.mean(fused_display[mid_row, :, :], axis=-1)

ax7.plot(profile_ms, label='MS', linewidth=2, color=COLORS['warning'], alpha=0.8)
ax7.plot(profile_fused, label='Fused', linewidth=2, color=COLORS['success'], alpha=0.8)
ax7.set_title(f'Horizontal Profile (Row {mid_row})', fontsize=11)
ax7.set_xlabel('Column')
ax7.set_ylabel('Mean Intensity')
ax7.legend()
ax7.grid(True, alpha=0.3)

# Metrics box
ax8.axis('off')
metrics = calculate_all_metrics(fused_display, ms_display_up)

metrics_text = f"""
╔═══════════════════════════════════╗
║     QUALITY METRICS SUMMARY       ║
╠═══════════════════════════════════╣
║  📈 PSNR:  {metrics['PSNR']:>8.2f} dB           ║
║  🎯 SSIM:  {metrics['SSIM']:>8.4f}              ║
║  🌈 SAM:   {metrics.get('SAM', 0):>8.2f}°              ║
╠═══════════════════════════════════╣
║  Input:  {str(ms_raw.shape):>22}  ║
║  Output: {str(fused_raw.shape):>22}  ║
╚═══════════════════════════════════╝
"""

ax8.text(0.5, 0.5, metrics_text, transform=ax8.transAxes,
         ha='center', va='center', fontsize=11, family='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor=COLORS['panel'],
                   edgecolor=COLORS['success'], linewidth=2))

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'detailed_analysis.png', dpi=150,
            facecolor=COLORS['bg'], bbox_inches='tight')
print("💾 Saved: results/detailed_analysis.png")
plt.show()

---
## 9. 🔍 Zoom Comparison

In [ ]:
apply_modern_style()

fig = plt.figure(figsize=(16, 10))
fig.suptitle('🔍 Zoom Comparison', fontsize=16, fontweight='bold',
             color=COLORS['success'])

# Get center region
h, w = fused_display.shape[:2]
crop_size = min(h, w) // 4
y1, y2 = h // 2 - crop_size, h // 2 + crop_size
x1, x2 = w // 2 - crop_size, w // 2 + crop_size

gs = GridSpec(2, 3, figure=fig, hspace=0.25, wspace=0.15)

# Full images (top)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])

# Zoomed (bottom)
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])

# Full images with rectangle
for ax, img, title, color in [
    (ax1, ms_display_up, 'MS', COLORS['ms_color']),
    (ax2, pan_display, 'PAN', COLORS['pan_color']),
    (ax3, fused_display, 'Fused', COLORS['success'])
]:
    if img.ndim == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)
    ax.set_title(title, fontsize=12, color=color)
    ax.axis('off')
    
    rect = mpatches.Rectangle((x1, y1), x2-x1, y2-y1,
                               linewidth=2, edgecolor=COLORS['highlight'],
                               facecolor='none')
    ax.add_patch(rect)

# Zoomed regions
for ax, img, title, color in [
    (ax4, ms_display_up[y1:y2, x1:x2], 'MS (Zoomed)', COLORS['ms_color']),
    (ax5, pan_display[y1:y2, x1:x2], 'PAN (Zoomed)', COLORS['pan_color']),
    (ax6, fused_display[y1:y2, x1:x2], 'Fused (Zoomed)', COLORS['success'])
]:
    if img.ndim == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)
    ax.set_title(title, fontsize=12, color=color, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'zoom_comparison.png', dpi=150,
            facecolor=COLORS['bg'], bbox_inches='tight')
print("💾 Saved: results/zoom_comparison.png")
plt.show()

---
## 10. 🎛️ Interactive Band Selector

In [ ]:
if HAS_WIDGETS and fused_raw.ndim == 3:
    n_bands = fused_raw.shape[-1]
    
    output = Output()
    
    def show_band(band_idx=0):
        with output:
            output.clear_output(wait=True)
            apply_modern_style()
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            fig.suptitle(f'Band {band_idx + 1} Comparison', fontsize=14,
                         color=COLORS['success'])
            
            # MS band (upsampled)
            ms_band_idx = min(band_idx, ms_raw.shape[-1] - 1)
            ms_band = zoom(ms_raw[..., ms_band_idx], 
                          (pan_raw.shape[0]/ms_raw.shape[0], 
                           pan_raw.shape[1]/ms_raw.shape[1]), order=1)
            ms_band = normalize_for_display(ms_band)
            
            axes[0].imshow(ms_band, cmap='viridis')
            axes[0].set_title(f'MS Band {ms_band_idx + 1}', color=COLORS['ms_color'])
            axes[0].axis('off')
            
            # PAN
            axes[1].imshow(pan_display, cmap='gray')
            axes[1].set_title('PAN', color=COLORS['pan_color'])
            axes[1].axis('off')
            
            # Fused band
            fused_band = normalize_for_display(fused_raw[..., band_idx])
            axes[2].imshow(fused_band, cmap='viridis')
            axes[2].set_title(f'Fused Band {band_idx + 1}', color=COLORS['success'])
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
    
    band_slider = IntSlider(value=0, min=0, max=n_bands-1, step=1,
                            description='Band:', style={'description_width': '60px'},
                            layout=widgets.Layout(width='60%'))
    
    interactive_bands = interactive(show_band, band_idx=band_slider)
    display(VBox([band_slider, output]))
    show_band(0)
else:
    print("⚠️ Interactive band selector requires ipywidgets and multi-band images")

---
## 11. 📋 Summary

In [ ]:
print("\n" + "="*60)
print("   📋 VISUALIZATION SUMMARY")
print("="*60)

print(f"\n📁 Input Images:")
print(f"   • MS:  {MS_PATH.name} - Shape: {ms_raw.shape}")
print(f"   • PAN: {PAN_PATH.name} - Shape: {pan_raw.shape}")

print(f"\n📊 Output:")
print(f"   • Fused Shape: {fused_raw.shape}")

print(f"\n📈 Quality Metrics:")
metrics = calculate_all_metrics(fused_display, ms_display_up)
print(f"   • PSNR: {metrics['PSNR']:.2f} dB")
print(f"   • SSIM: {metrics['SSIM']:.4f}")
if 'SAM' in metrics:
    print(f"   • SAM:  {metrics['SAM']:.2f}°")

print(f"\n💾 Saved Figures:")
print(f"   • results/comparison_modern.png")
print(f"   • results/detailed_analysis.png")
print(f"   • results/zoom_comparison.png")

print("\n" + "="*60)
print("   ✅ Visualization Complete!")
print("="*60)